In [ ]:
!pip install medmnist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
os.makedirs("images", exist_ok=True)

n_epochs = 200
batch_size = 256
lr = 0.001
b1 = 0.5
b2 = 0.999
n_cpu = -1
latent_dim = 100
n_classes = 8
img_size = 28
channels = 1
sample_interval = 400

cuda = True if torch.cuda.is_available() else False


In [ ]:
import os
import torch
from torchvision import datasets, transforms
from medmnist import dataset
from medmnist import INFO

In [ ]:
torch.manual_seed(1312)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.4
                                                                                                                )]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
            *discriminator_block(128, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4

        # self.tmp = ds_size
        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        # print(self.tmp)
        # print(128 * self.tmp ** 2)
        # print(out.shape)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label


# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
data_dir = "../../data/medmnist"  # Path to the MedMNIST dataset directory
os.makedirs(data_dir, exist_ok=True)
# dataloader = torch.utils.data.DataLoader(datasets.MNIST(
#     "../../data/mnist",
#     train=True,
#     download=True,
#     transform=transforms.Compose(
#         [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#     ),
# ),
#     batch_size=batch_size,
#     shuffle=True,
# )
dataloader = torch.utils.data.DataLoader(
    dataset.TissueMNIST(
        # root=data_dir,
        split='train',
        transform=transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ]),
        target_transform=transforms.Lambda(lambda x: x[0]),
        # flag=dataset_name,
        download=True
    ),
    batch_size=batch_size,
    shuffle=True
)

dataloader_test = torch.utils.data.DataLoader(
    dataset.TissueMNIST(
        # root=data_dir,
        split='test',
        transform=transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ]),
        target_transform=transforms.Lambda(lambda x: x[0]),
        # flag=dataset_name,
        download=True
    ),
    batch_size=batch_size,
    shuffle=True
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

scheduler_G = torch.optim.lr_scheduler.StepLR(optimizer_G, step_size=len(dataloader)*5, gamma=0.5, last_epoch=- 1, verbose=False)
scheduler_D = torch.optim.lr_scheduler.StepLR(optimizer_D, step_size=len(dataloader)*5, gamma=0.5, last_epoch=- 1, verbose=False)

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


Using downloaded and verified file: /root/.medmnist/tissuemnist.npz
Using downloaded and verified file: /root/.medmnist/tissuemnist.npz


In [ ]:
# ----------
#  Training
# ----------

for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):
        generator.train()
        discriminator.train()
        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()
        # scheduler_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()
        scheduler_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()
        # scheduler_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        # fake_pred, fake_aux = discriminator(gen_imgs.detach())
        # d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        # d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss = d_real_loss

        if i % 50 == 0:
            # Calculate discriminator accuracy
            pred = np.concatenate([real_aux.data.cpu().numpy()], axis=0)
            # pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
            gt = np.concatenate([labels.data.cpu().numpy()], axis=0)
            # gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
            d_acc = np.mean(np.argmax(pred, axis=1) == gt)
            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
                % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
            )

        d_loss.backward()
        optimizer_D.step()
        scheduler_D.step()

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=n_classes, batches_done=batches_done)

    avg_acc = 0
    avg_real_acc = 0
    avg_fake_acc = 0

    for i, (imgs, labels) in enumerate(dataloader_test):
        generator.eval()
        discriminator.eval()
        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Test Generator
        # -----------------

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)


        # ---------------------
        #  Test Discriminator
        # ---------------------

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)
        avg_acc += d_acc

        avg_real_acc += np.mean(np.argmax(real_aux.data.cpu().numpy(), axis=1) == labels.data.cpu().numpy())
        avg_fake_acc += np.mean(np.argmax(fake_aux.data.cpu().numpy(), axis=1) == gen_labels.data.cpu().numpy())

        # print(
        #     "TEST [Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
        #     % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
        # )
        batches_done = epoch * len(dataloader_test) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=n_classes, batches_done=batches_done)

    print("TEST acc: %d%%" % (100 * avg_acc / len(dataloader)))
    print("TEST real acc: %d%%" % (100 * avg_real_acc / len(dataloader)))
    print("TEST fake acc: %d%%" % (100 * avg_fake_acc / len(dataloader)))
    print("LR: ", scheduler_D.get_last_lr(), scheduler_G.get_last_lr())


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 0/200] [Batch 0/647] [D loss: 1.371602, acc: 8%] [G loss: 1.371570]
[Epoch 0/200] [Batch 50/647] [D loss: 1.149022, acc: 36%] [G loss: 1.168343]
[Epoch 0/200] [Batch 100/647] [D loss: 0.985801, acc: 44%] [G loss: 1.079924]
[Epoch 0/200] [Batch 150/647] [D loss: 0.963104, acc: 38%] [G loss: 1.019454]
[Epoch 0/200] [Batch 200/647] [D loss: 0.948442, acc: 39%] [G loss: 1.016670]
[Epoch 0/200] [Batch 250/647] [D loss: 0.911659, acc: 46%] [G loss: 0.990356]
[Epoch 0/200] [Batch 300/647] [D loss: 0.908557, acc: 47%] [G loss: 0.952229]
[Epoch 0/200] [Batch 350/647] [D loss: 0.922223, acc: 42%] [G loss: 0.953874]
[Epoch 0/200] [Batch 400/647] [D loss: 0.917200, acc: 43%] [G loss: 0.969383]
[Epoch 0/200] [Batch 450/647] [D loss: 0.908202, acc: 46%] [G loss: 0.937334]
[Epoch 0/200] [Batch 500/647] [D loss: 0.906127, acc: 44%] [G loss: 0.915776]
[Epoch 0/200] [Batch 550/647] [D loss: 0.926643, acc: 42%] [G loss: 0.898397]
[Epoch 0/200] [Batch 600/647] [D loss: 0.886669, acc: 50%] [G loss: 